In [1]:
import folium
import pandas as pd
import pickle
import os
import h3
from label_samples_time_hexa import label_samples
from vizualize import draw_hexagons, draw_migration_for_time_bin, draw_hexagons_with_values, draw_barriers
from func import rename_time_bins, calc_dist_time_bin, normalize_distances, get_time_bin_hexagons, get_min_max_dist, get_isolated_hex_and_barriers, find_closest_population

In [2]:
matrix = pd.read_pickle("/home/jaro/BINP29/Project_Eran/1_dist_matrix/eucl_dist.pkl")

In [3]:
time_bins = 18
resolution = 3
same_age_range = True
df = label_samples("/home/jaro/BINP29/Project_Eran/", time_bins, resolution, same_age_range)
time_bins = rename_time_bins(df)
time_bins_hexagons = get_time_bin_hexagons(df)

In [4]:
# calculate the average distances between neighboring hexagons for each time bin with the given parameters
time_bins_dist = calc_dist_time_bin(df, matrix,
                                        10,
                                        False)

In [5]:
selected_time_bin = time_bins[2]
hexagons = time_bins_hexagons[selected_time_bin]
time_bin = time_bins_dist[selected_time_bin]
time_bin = normalize_distances(time_bin)
threshold = 0.4

In [6]:
isolated_hex, barrier_lines, barrier_hex = get_isolated_hex_and_barriers(time_bin, hexagons, threshold)

print(f"Number of isolated hexagons: {len(isolated_hex)}")
print(f"Number of barrier lines: {len(barrier_lines)}")
print(f"Number of barrier hexagons: {len(barrier_hex)}")

Number of isolated hexagons: 8
Number of barrier lines: 206
Number of barrier hexagons: 1328


In [7]:
# get the index of the time bin of interest
time_bin = time_bins.index(time_bins[2])
# get the closest population for each isolated hexagon
closest_populations, new_isolated_hex = find_closest_population(df, time_bin, isolated_hex, matrix, threshold)
print(f"Number of isolated hexagons with no migration: {len(new_isolated_hex)}")

Number of isolated hexagons with no migration: 4


In [11]:
m = draw_hexagons(hexagons, color = "darkgreen")
m = draw_hexagons(new_isolated_hex, color = "red", m = m, opacity=0.6, value="No migration")
m = draw_migration_for_time_bin(closest_populations, m)
m = draw_hexagons_with_values(barrier_hex, m, threshold = 0.0)
m = draw_barriers(barrier_lines, m)
m

In [17]:
# create two hexagons using h3
hex1 = h3.geo_to_h3(37.3615593, -122.0553238, 3)
hex2 = h3.geo_to_h3(36.3615593, -122.0553238, 3)
print(h3.distance(hex1, hex2))

hex1_dots = h3.h3_to_geo_boundary(hex1)
hex2_dots = h3.h3_to_geo_boundary(hex2)
# get the pair of dots that the two hexagons share
dot = [x for x in hex1_dots if x in hex2_dots]
dot
# m = draw_hexagons([hex1, hex2], color = 'blue')
# # draw the line between the two hexagons
# folium.PolyLine(dot, color = 'red').add_to(m)
# m




AttributeError: module 'h3' has no attribute 'distance'